In [1]:
from pyomo_orm.core.database import Session, bind_engine
from pyomo_orm.core.utils import as_dataframe

from models import *

In [2]:
bind_engine('sqlite:///diet.sqlite')

Engine(sqlite:///diet.sqlite)

In [3]:
from pyomo.environ import AbstractModel, Param, PositiveReals, NonNegativeIntegers, Objective, Constraint, NonNegativeReals

diet = AbstractModel()
infinity = float('inf')

In [4]:
# Sets
diet.food = Food.create_set()
diet.nutrients = Nutrient.create_set()

In [5]:
# index params
diet.cost = Food.create_param(diet, from_attr='cost', indexed_by='id')
diet.volume_per_serving = Food.create_param(diet, from_attr='volume_per_serving', indexed_by='id')
diet.nutrient_lower_bound = Nutrient.create_param(diet, from_attr='lower_bound', indexed_by='id', within=NonNegativeReals, default=0.0)
diet.nutrient_upper_bound = Nutrient.create_param(diet, from_attr='upper_bound', indexed_by='id', within=NonNegativeReals, default=infinity)

In [6]:
# multiindex params
diet.nutrient_amount = FoodNutrientAmount.create_param(diet, from_attr='amount', indexed_by=('food_id', 'nutrient_id'))

In [7]:
diet.volume_max = Param(within=PositiveReals)

In [8]:
# variable
diet.amount_in_diet = Food.create_var(diet, from_attr='amount_in_diet', indexed_by='id', within=NonNegativeIntegers)

In [9]:
# constraints
def cost_rule(m):
    return (sum(m.cost[i] * m.amount_in_diet[i] for i in m.food))
diet.objective = Objective(rule=cost_rule)

def nutrient_rule_lower(m, j):
    value = sum(m.nutrient_amount[i, j] * m.amount_in_diet[i] for i in m.food)
    return m.nutrient_lower_bound[j] <= value

def nutrient_rule_upper(m,j):
    value = sum(m.nutrient_amount[i, j] * m.amount_in_diet[i] for i in m.food)
    return value <= m.nutrient_upper_bound[j]
diet.nutrient_constraint_lower = Constraint(diet.nutrients, rule=nutrient_rule_lower)
diet.nutrient_constraint_upper = Constraint(diet.nutrients, rule=nutrient_rule_upper)

def volume_rule(m):
    return sum(m.volume_per_serving[i] * m.amount_in_diet[i] for i in m.food) <= m.volume_max
diet.volume_constraint = Constraint(rule=volume_rule)

In [10]:
from pyomo_orm.core.utils import create_data_dict

di = create_data_dict(diet, 'diet')
di['diet']['volume_max'] = {None: 75.0}

In [11]:
di

{'diet': {'cost': {1: 1.84,
   2: 2.19,
   3: 1.84,
   4: 1.44,
   5: 2.29,
   6: 0.77,
   7: 1.29,
   8: 0.6,
   9: 0.72},
  'food': {None: [1, 2, 3, 4, 5, 6, 7, 8, 9]},
  'nutrient_amount': {(1, 1): 510.0,
   (1, 2): 34.0,
   (1, 3): 28.0,
   (1, 4): 15.0,
   (1, 5): 6.0,
   (1, 6): 30.0,
   (1, 7): 20.0,
   (2, 1): 370.0,
   (2, 2): 35.0,
   (2, 3): 24.0,
   (2, 4): 15.0,
   (2, 5): 10.0,
   (2, 6): 20.0,
   (2, 7): 20.0,
   (3, 1): 500.0,
   (3, 2): 42.0,
   (3, 3): 25.0,
   (3, 4): 6.0,
   (3, 5): 2.0,
   (3, 6): 25.0,
   (3, 7): 20.0,
   (4, 1): 370.0,
   (4, 2): 38.0,
   (4, 3): 14.0,
   (4, 4): 2.0,
   (4, 5): 0.0,
   (4, 6): 15.0,
   (4, 7): 10.0,
   (5, 1): 400.0,
   (5, 2): 42.0,
   (5, 3): 31.0,
   (5, 4): 8.0,
   (5, 5): 15.0,
   (5, 6): 15.0,
   (5, 7): 8.0,
   (6, 1): 220.0,
   (6, 2): 26.0,
   (6, 3): 3.0,
   (6, 4): 0.0,
   (6, 5): 15.0,
   (6, 6): 0.0,
   (6, 7): 2.0,
   (7, 1): 345.0,
   (7, 2): 27.0,
   (7, 3): 15.0,
   (7, 4): 4.0,
   (7, 5): 0.0,
   (7, 6): 20.0,


In [12]:
from pyomo.environ import SolverFactory

instance = diet.create_instance(data=di, namespace='diet')
solver = SolverFactory('cbc')
results = solver.solve(instance, tee=True)

Welcome to the CBC MILP Solver 
Version: 2.9.7 
Build Date: Oct 10 2015 
Revision Number: 2226 

command line - C:\Program Files (x86)\COIN-OR\1.8.0\win32-msvc12\bin\cbc.exe -printingOptions all -import C:\Users\tom\AppData\Local\Temp\tmpk1jdj5qf.pyomo.lp -stat=1 -solve -solu C:\Users\tom\AppData\Local\Temp\tmpk1jdj5qf.pyomo.soln (default strategy 1)
Option for printingOptions changed from normal to all
Presolve 9 (-1) rows, 9 (-1) columns and 76 (-1) elements
Statistics for presolved model
Original problem has 9 integers (0 of which binary)
Presolved problem has 9 integers (0 of which binary)
==== 0 zero objective 8 different
1 variables have objective of 0.6
1 variables have objective of 0.72
1 variables have objective of 0.77
1 variables have objective of 1.29
1 variables have objective of 1.44
2 variables have objective of 1.84
1 variables have objective of 2.19
1 variables have objective of 2.29
==== absolute objective values 8 different
1 variables have objective of 0.6
1 variabl

In [13]:
as_dataframe(Food.query())

,id,name,cost,volume_per_serving,amount_in_diet,model_run_id
0,1,Cheeseburger,1.84,4.0,None,None
1,2,Ham Sandwich,2.19,7.5,None,None
2,3,Hamburger,1.84,3.5,None,None
3,4,Fish Sandwich,1.44,5.0,None,None
4,5,Chicken Sandwich,2.29,7.3,None,None
5,6,Fries,0.77,2.6,None,None
6,7,Sausage Biscuit,1.29,4.1,None,None
7,8,Lowfat Milk,0.60,8.0,None,None
8,9,Orange Juice,0.72,12.0,None,None


In [14]:
instance.amount_in_diet.extract_values()

{1: 4.0, 2: 0.0, 3: 0.0, 4: 1.0, 5: 0.0, 6: 5.0, 7: 0.0, 8: 4.0, 9: 0.0}

In [15]:
import pandas as pd
pd.Series(instance.amount_in_diet.extract_values())

1    4.0
2    0.0
3    0.0
4    1.0
5    0.0
6    5.0
7    0.0
8    4.0
9    0.0
dtype: float64

In [16]:
instance.write()

('unknown.lp', 2539451803856)

In [36]:
Food.query().filter(Food.id.in_(list(di['diet']['cost'].keys()))).all()

[<Food: Cheeseburger>,
 <Food: Ham Sandwich>,
 <Food: Hamburger>,
 <Food: Fish Sandwich>,
 <Food: Chicken Sandwich>,
 <Food: Fries>,
 <Food: Sausage Biscuit>,
 <Food: Lowfat Milk>,
 <Food: Orange Juice>]

In [47]:
list(instance.food)

[1, 2, 3, 4, 5, 6, 7, 8, 9]

In [46]:
list(instance.cost)

[1, 2, 3, 4, 5, 6, 7, 8, 9]

In [55]:
Food.__name__

'Food'